# Perform lever-arm correction to all sensors from Sentry Nav

Sentry measured rotations relate directly to local spike translations.
- Pitching motion (about Sentry y-axis) relates to vertical motions in the spike.
- Bottom following autonomous mode will result in many pitching adjustments
 - Likely a major vertical component measured in ADV 

In [ ]:
# Libraries for plotting
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load processed Nav data
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_converted.h5', 'table')

In [ ]:
# get start time
T0 = nav.head(1).epoch[0]

# see what is in the data
nav.head(2)

## Lever-arm uses the relative position from reference to test point

In [ ]:
# Sentry to spike GX3-25,  this is the guesstimate, will look up more accurate number 
bTRgxs = np.array([1.8, 0.2, 0.7])
# Sentry to body GX3-25,  also a guesstimate
bTRgxb = np.array([0.0, 0.0, 0.0])
# Sentry to ADV,  also a guesstimate
bTRadv = np.array([1.6, 0.2, 0.9])
# Sentry to SBE3,  also a guesstimate
bTRsbe = np.array([1.5, 0.2, 0.8])
# Sentry to Paros,  also a guesstimate
bTRpar = np.array([1.4, 0.2, 0.8])

## Extract time and pitch

Convert pitch from degrees to radians

In [ ]:
# Data for plotting
t = nav.epoch - T0
pitch = nav.pitch * np.pi/180.0

### Double checking epoch time?

There is a jump in timestamp?

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(len(t)),t)

ax.set(xlabel='samples', ylabel='epoch time',
       title='Data from two dives?')
ax.grid()
fig.savefig('../results/nav/epoch-time.png')

## Working with a shorter subset of data

In [ ]:
strt = 0
ephs = 65000

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], pitch[strt:ephs])

ax.set(xlabel='time [s]', ylabel='pitch [rad]',
       title='Pitch angle')
ax.grid()
fig.savefig('../results/nav/pitch_vs_time.png')

# Calculate lever-arm perturbations to Spike mounted sensors

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], nav.head(ephs).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth over time')
ax.grid()
fig.savefig('../results/nav/depth_vs_time.png')

## Seems to be some erroneous data between 65000 and 70000

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[65000:70000], nav.head(70000).tail(5000).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth Measurement Error')
ax.grid()
fig.savefig('../results/nav/depth_data_err.png')

## Zoom in closer on the first dive

In [ ]:
strt = 14000
ephs = 50500

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], nav.head(ephs).tail(ephs-strt).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth over time')
ax.grid()
fig.savefig('../results/nav/depth_vs_time_bottom1_all.png')

In [ ]:
strt = 14780
ephs = 25330

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], nav.head(ephs).tail(ephs-strt).depth)

ax.set(xlabel='time [s]', ylabel='depth [m]',
       title='Depth Zoomed, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/depth_vs_time_bottom1_1480.png')

In [ ]:
fig, ax = plt.subplots()
ax.plot(t[strt:ephs], pitch[strt:ephs])

ax.set(xlabel='time [s]', ylabel='pitch [rad]',
       title='Pitch angle, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/pitch_vs_time_bottom1_1480.png')

### What are the Local Vertical Perturbations due to Pitching

In [ ]:
# focus on just pitch for the moment
local_pitch = np.array(pitch[strt:ephs])
ref_pitch = np.mean(local_pitch)
delta_pitch = local_pitch - ref_pitch

# calculate the delta time
local_time = np.array(t[strt:ephs])
delta_time = np.diff(local_time)

# pitch rate
local_pitch_rate = delta_pitch / np.mean(delta_time)

In [ ]:
fig, ax = plt.subplots()
ax.plot(local_pitch_rate)

ax.set(xlabel='time [s]', ylabel='pitch rate [rad/s]',
       title='Pitch rate, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/pitchrate_vs_time_bottom1_1480.png')

In [ ]:
# calculate the lever-arm component of vertical velocity

# result
res = np.matrix((10550,3))
vert = np.zeros((10550,))

vert.shape

In [ ]:
for i in range(10550):
  local_rate = np.array([0.0, local_pitch_rate[i], 0.0])
  val = np.cross(local_rate, bTRadv)
  vert[i] = val[2]

In [ ]:
fig, ax = plt.subplots()
ax.plot(vert)

ax.set(xlabel='time [s]', ylabel='Vertical Vel [ms/s]',
       title='Vertical Velocity, Bottom1, -1480m')
ax.grid()
fig.savefig('../results/nav/vert_vs_time_bottom1_1480.png')